### From MLG to nn

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn


In [3]:
df  = datasets.load_diabetes()

In [4]:
X = torch.from_numpy(df.data)
y = torch.from_numpy(df.target)

### Nothing new until now

In [5]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(X, y)

from torch.utils.data import DataLoader


train_loader = DataLoader(dataset, batch_size = 25, shuffle=True)


In [34]:
#https://www.analyticsvidhya.com/blog/2021/08/linear-regression-and-gradient-descent-in-pytorch/
def rmse(y,y_hat):
    return torch.sqrt(((torch.pow(y-y_hat,2))/torch.numel(y)).sum())



In [6]:
w = torch.randn(1,int(X[1].numel()), requires_grad=True)
b = torch.randn(int(X.numel()/X[1].numel()), 1, requires_grad=True)


In [12]:
def LinearReg(X):
    y = X.float() @ w.transpose(0,1) + b
    return y
    

In [35]:
#LinearReg(X)

mse(y, LinearReg(X))

tensor(12857710.4211, dtype=torch.float64, grad_fn=<SumBackward0>)

AttributeError: 'NoneType' object has no attribute 'zero_'

In [69]:
learning_rate = 0.01
for i in range(1000):
    y_hat = LinearReg(train_loader)
    loss = mse(y,y_hat)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad = None
        b.grad = None

In [70]:
rmse(y, LinearReg(X))

tensor(1618.9531, dtype=torch.float64, grad_fn=<SqrtBackward>)

In [60]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(df.data,df.target)

In [62]:
from sklearn.metrics import mean_squared_error

mean_squared_error(reg.predict(df.data),df.target)

2859.6903987680657

In [92]:
class LinearReg():
   def __init__(self):
      self.w = torch.randn(1,int(X[1].numel()), requires_grad=True)
      self.b = torch.randn(int(X.numel()/X[1].numel()), 1, requires_grad=True)
   def LinearReg(X):
      y = X.float() @ self.w.transpose(0,1) + self.b
      return self.y

   def rmse(self,y,y_hat):
      return torch.sqrt(((torch.pow(self.y-self.y_hat,2))/torch.numel(self.y)).sum())

   def fit(self,X):
      learning_rate = 0.01
      for i in range(300):
         y_hat = LinearReg(X)
         loss = rmse(self.y,y_hat)
         loss.backward()
         with torch.no_grad():
            w -= learning_rate * self.w.grad
            b -= learning_rate * self.b.grad
            self.w.grad = None
            self.b.grad = None


In [95]:
x = LinearReg()
x.LinearReg()


AttributeError: 'LinearReg' object has no attribute 'float'